# Microsoft Fabric Capacity Migration Discovery

## Overview
This notebook performs comprehensive discovery of Power BI Premium/Embedded capacities and identifies potential blockers for migration to Microsoft Fabric capacities.

## What This Notebook Does
1. ✅ Validates prerequisites (admin access, permissions)
2. 📊 Collects data from all capacities, workspaces, and items
3. 🔍 Analyzes migration blockers and compatibility issues
4. 💾 Stores data in a Fabric Lakehouse
5. 📈 Creates interactive Power BI report for analysis

## Prerequisites
- **Tenant Administrator** or **Capacity Administrator** permissions
- XMLA Read/Write enabled on capacities
- Fabric workspace with Lakehouse creation permissions

## Last Updated
November 2025

## Step 1: Install Required Libraries

Install semantic-link and semantic-link-labs for Fabric integration.

In [ ]:
%pip install semantic-link-labs --upgrade --quiet
print("✓ Libraries installed successfully")

## Step 2: Configure Parameters

Set the names for your lakehouse, semantic model, and report.

In [ ]:
# Configuration Parameters
semantic_model_name = "Capacity Migration Analysis"
report_name = "Capacity Migration Report"
lakehouse = "CapacityMigrationLH"

# Validate parameters
if not all([semantic_model_name, report_name, lakehouse]):
    raise ValueError("❌ Error: All parameters (semantic_model_name, report_name, lakehouse) must be set")

print("✓ Configuration validated:")
print(f"  • Lakehouse: {lakehouse}")
print(f"  • Semantic Model: {semantic_model_name}")
print(f"  • Report: {report_name}")

## Step 3: Import Libraries and Validate Permissions

In [ ]:
import sempy
import sempy.fabric as fabric
import sempy_labs as labs
import sempy_labs.admin as labs_admin
import sempy_labs.report as labs_report
import pandas as pd
import json
from datetime import datetime

print("✓ Libraries imported successfully")

# Validate tenant admin access
try:
    test_capacities = labs_admin._capacities._list_capacities_meta()
    print(f"✓ Tenant admin access confirmed")
    print(f"  • Found {len(test_capacities)} capacities accessible")
except Exception as e:
    print(f"❌ Error: Requires tenant/capacity admin permissions")
    print(f"  • Details: {str(e)}")
    raise

## Step 4: Create or Verify Lakehouse

In [ ]:
# Check if Lakehouse exists, otherwise create it
try:
    df_items = fabric.list_items()
    lakehouse_exists = df_items['Display Name'].eq(lakehouse).any()
    
    if lakehouse_exists:
        lhid = df_items[
            (df_items['Display Name'].eq(lakehouse)) & 
            (df_items['Type'].eq('Lakehouse'))
        ].iloc[0, 0]
        print(f"✓ Lakehouse '{lakehouse}' already exists (ID: {lhid})")
    else:
        print(f"Creating new Lakehouse '{lakehouse}'...")
        lhid = fabric.create_lakehouse(lakehouse)
        print(f"✓ Lakehouse created successfully (ID: {lhid})")
except Exception as e:
    print(f"❌ Error creating/verifying Lakehouse: {str(e)}")
    raise

## Step 5: Collect Capacity Data

Gathering all capacity information including SKU, region, and state.

In [ ]:
print("Collecting capacity data...")

try:
    # Get all capacities
    df_capacities = labs_admin._capacities._list_capacities_meta()
    
    # Convert Admins column to string to avoid type conflicts
    if 'Admins' in df_capacities.columns:
        df_capacities['Admins'] = df_capacities['Admins'].apply(
            lambda x: json.dumps(x) if x is not None else None
        )
    
    # Add a record for non-premium workspaces
    new_record = pd.DataFrame([{
        "Capacity Id": "-1",
        "Capacity Name": "Non Premium (Shared)",
        "Sku": "Shared",
        "Region": "N/A",
        "State": "Active",
        "Admins": json.dumps(["N/A"])
    }])
    
    df_capacities = pd.concat([df_capacities, new_record], ignore_index=True)
    
    print(f"✓ Collected {len(df_capacities) - 1} Premium/Fabric capacities")
    print(f"\nCapacity Summary:")
    print(df_capacities.groupby('Sku').size().to_string())
    
    # Save to Lakehouse
    labs.save_as_delta_table(
        dataframe=df_capacities,
        delta_table_name="Capacities",
        write_mode="overwrite",
        lakehouse=lakehouse
    )
    print("\n✓ Capacities data saved to Lakehouse")
    
except Exception as e:
    print(f"❌ Error collecting capacity data: {str(e)}")
    raise

## Step 6: Collect Workspace Data

Gathering all workspace information and capacity assignments.

In [ ]:
print("Collecting workspace data...")

try:
    # Get all workspaces
    all_workspaces_df = labs_admin.list_workspaces()
    
    # Fill null Capacity Ids with -1 (non-premium)
    all_workspaces_df['Capacity Id'] = all_workspaces_df['Capacity Id'].fillna("-1")
    
    print(f"✓ Collected {len(all_workspaces_df)} workspaces")
    
    # Workspace breakdown
    premium_count = (all_workspaces_df['Capacity Id'] != "-1").sum()
    shared_count = (all_workspaces_df['Capacity Id'] == "-1").sum()
    
    print(f"  • Premium/Fabric workspaces: {premium_count}")
    print(f"  • Shared workspaces: {shared_count}")
    
    # Save to Lakehouse
    labs.save_as_delta_table(
        dataframe=all_workspaces_df,
        delta_table_name="Workspaces",
        write_mode="overwrite",
        lakehouse=lakehouse
    )
    print("✓ Workspaces data saved to Lakehouse")
    
except Exception as e:
    print(f"❌ Error collecting workspace data: {str(e)}")
    raise

## Step 7: Collect Workspace Items

Gathering all items (reports, datasets, dataflows, etc.) across all workspaces.

In [ ]:
print("Collecting workspace items...")

try:
    # Get all items across all workspaces
    df_items = labs_admin.list_items()
    
    print(f"✓ Collected {len(df_items)} workspace items")
    print(f"\nItem Type Breakdown:")
    print(df_items['Type'].value_counts().to_string())
    
    # Save to Lakehouse
    labs.save_as_delta_table(
        dataframe=df_items,
        delta_table_name="WorkspaceItems",
        write_mode="overwrite",
        lakehouse=lakehouse
    )
    print("\n✓ Workspace items data saved to Lakehouse")
    
except Exception as e:
    print(f"❌ Error collecting workspace items: {str(e)}")
    raise

## Step 8: Collect Semantic Model Data

Gathering detailed information about all semantic models (datasets).

In [ ]:
print("Collecting semantic model data...")

try:
    # Get all semantic models
    df_semantic_models = labs_admin.list_datasets()
    
    # Convert complex fields to JSON strings for storage
    df_semantic_models["Upstream Datasets"] = df_semantic_models["Upstream Datasets"].apply(
        lambda x: json.dumps(x) if x and x != "[]" else None
    )
    df_semantic_models["Users"] = df_semantic_models["Users"].apply(
        lambda x: json.dumps(x) if x and x != "[]" else None
    )
    
    print(f"✓ Collected {len(df_semantic_models)} semantic models")
    
    # Storage mode breakdown
    if 'Target Storage Mode' in df_semantic_models.columns:
        print(f"\nStorage Mode Breakdown:")
        print(df_semantic_models['Target Storage Mode'].value_counts().to_string())
    
    # Save to Lakehouse
    labs.save_as_delta_table(
        dataframe=df_semantic_models,
        delta_table_name="SemanticModels",
        write_mode="overwrite",
        lakehouse=lakehouse
    )
    print("\n✓ Semantic models data saved to Lakehouse")
    
except Exception as e:
    print(f"❌ Error collecting semantic model data: {str(e)}")
    raise

## Step 9: Migration Blocker Analysis

Analyzing potential blockers for migration from Premium to Fabric capacities.

In [ ]:
print("=" * 70)
print("MIGRATION READINESS ASSESSMENT")
print("=" * 70)
print()

blockers = []
warnings = []
info_items = []

# ============================================
# 1. Check for Embedded (EM) SKUs
# ============================================
em_skus = df_capacities[
    df_capacities['Sku'].str.startswith('EM', na=False) |
    df_capacities['Sku'].str.startswith('A', na=False)  # Azure A SKUs
]
if not em_skus.empty:
    for _, cap in em_skus.iterrows():
        if cap['Sku'].startswith('EM'):
            blockers.append(
                f"Embedded capacity '{cap['Capacity Name']}' (SKU: {cap['Sku']}) - "
                f"EM SKUs not supported in Fabric. Migrate to F-SKUs."
            )
        elif cap['Sku'].startswith('A'):
            info_items.append(
                f"Azure capacity '{cap['Capacity Name']}' (SKU: {cap['Sku']}) - "
                f"Azure Embedded can migrate to Fabric F-SKUs."
            )

# ============================================
# 2. Check for Premium P SKUs (ready for migration)
# ============================================
p_skus = df_capacities[df_capacities['Sku'].str.startswith('P', na=False)]
if not p_skus.empty:
    info_items.append(
        f"{len(p_skus)} Premium P-SKU capacities found - Ready for Fabric migration"
    )

# ============================================
# 3. Check for cross-region workspaces
# ============================================
capacity_regions = df_capacities[
    df_capacities['Region'] != 'N/A'
]['Region'].unique()

if len(capacity_regions) > 1:
    warnings.append(
        f"Multiple regions detected: {', '.join(capacity_regions)}. "
        f"Fabric capacities are region-specific. Plan migrations within same region."
    )

# ============================================
# 4. Check for Dataflows Gen1
# ============================================
dataflows_gen1 = df_items[df_items['Type'] == 'Dataflow']
if not dataflows_gen1.empty:
    warnings.append(
        f"{len(dataflows_gen1)} Dataflow Gen1 artifacts found. "
        f"Consider upgrading to Dataflow Gen2 or Data Pipelines in Fabric."
    )

# ============================================
# 5. Check for Paginated Reports
# ============================================
paginated = df_items[df_items['Type'] == 'PaginatedReport']
if not paginated.empty:
    warnings.append(
        f"{len(paginated)} Paginated Reports found. "
        f"Ensure Fabric capacity has paginated report workload enabled."
    )

# ============================================
# 6. Check for Large Models (Premium Files)
# ============================================
if 'Target Storage Mode' in df_semantic_models.columns:
    large_models = df_semantic_models[
        df_semantic_models['Target Storage Mode'] == 'PremiumFiles'
    ]
    if not large_models.empty:
        warnings.append(
            f"{len(large_models)} Large Models (>10GB) detected. "
            f"Verify target Fabric capacity size supports these models."
        )

# ============================================
# 7. Check for Inactive Workspaces
# ============================================
if 'State' in all_workspaces_df.columns:
    inactive = all_workspaces_df[all_workspaces_df['State'] != 'Active']
    if not inactive.empty:
        warnings.append(
            f"{len(inactive)} inactive workspaces found (States: {inactive['State'].unique()}). "
            f"Review and clean up before migration."
        )

# ============================================
# 8. Check for Models with RLS
# ============================================
if 'Is Effective Identity Required' in df_semantic_models.columns:
    models_with_rls = df_semantic_models[
        df_semantic_models['Is Effective Identity Required'] == True
    ]
    if not models_with_rls.empty:
        info_items.append(
            f"{len(models_with_rls)} semantic models with Row-Level Security (RLS). "
            f"Test RLS behavior after migration, especially with DirectLake."
        )

# ============================================
# 9. Check for Dashboards
# ============================================
dashboards = df_items[df_items['Type'] == 'Dashboard']
if not dashboards.empty:
    info_items.append(
        f"{len(dashboards)} Dashboards found. Dashboards migrate with their tiles and data sources."
    )

# ============================================
# 10. Check for Deployment Pipelines
# ============================================
if 'Pipeline Id' in all_workspaces_df.columns:
    pipelines = all_workspaces_df[all_workspaces_df['Pipeline Id'].notna()]
    if not pipelines.empty:
        info_items.append(
            f"{len(pipelines)} workspaces use Deployment Pipelines. "
            f"Pipelines are supported in Fabric - verify configuration post-migration."
        )

# ============================================
# Display Results
# ============================================
print("\n")

if blockers:
    print("🛑 CRITICAL BLOCKERS (Must resolve before migration):")
    print("=" * 70)
    for i, blocker in enumerate(blockers, 1):
        print(f"{i}. {blocker}")
    print()

if warnings:
    print("⚠️  WARNINGS (Review and plan accordingly):")
    print("=" * 70)
    for i, warning in enumerate(warnings, 1):
        print(f"{i}. {warning}")
    print()

if info_items:
    print("ℹ️  INFORMATIONAL (For your awareness):")
    print("=" * 70)
    for i, info in enumerate(info_items, 1):
        print(f"{i}. {info}")
    print()

if not blockers and not warnings:
    print("✅ MIGRATION READY")
    print("=" * 70)
    print("No critical blockers or warnings detected.")
    print("Your environment appears ready for Fabric migration.")
    print()

# ============================================
# Save Analysis Results
# ============================================
analysis_results = pd.DataFrame({
    'Analysis Date': [datetime.now()],
    'Total Capacities': [len(df_capacities) - 1],
    'Total Workspaces': [len(all_workspaces_df)],
    'Total Items': [len(df_items)],
    'Total Models': [len(df_semantic_models)],
    'Blocker Count': [len(blockers)],
    'Warning Count': [len(warnings)],
    'Blockers': [json.dumps(blockers)],
    'Warnings': [json.dumps(warnings)],
    'Info': [json.dumps(info_items)]
})

labs.save_as_delta_table(
    dataframe=analysis_results,
    delta_table_name="MigrationAnalysis",
    write_mode="overwrite",
    lakehouse=lakehouse
)

print("✓ Analysis results saved to Lakehouse (Table: MigrationAnalysis)")
print("=" * 70)

## Step 10: Create Semantic Model

Creating a DirectLake semantic model for analysis and reporting.

In [ ]:
print("Creating semantic model...")

# Define the BIM (Business Intelligence Model) structure
bim_model = {
    "compatibilityLevel": 1604,
    "model": {
        "culture": "en-US",
        "dataAccessOptions": {
            "legacyRedirects": True,
            "returnErrorValuesAsNull": True
        },
        "defaultPowerBIDataSourceVersion": "powerBI_V3",
        "sourceQueryCulture": "en-US",
        "tables": [
            {
                "name": "Capacities",
                "columns": [
                    {"name": "Capacity Id", "dataType": "string", "sourceColumn": "Capacity Id"},
                    {"name": "Capacity Name", "dataType": "string", "sourceColumn": "Capacity Name"},
                    {"name": "Sku", "dataType": "string", "sourceColumn": "Sku"},
                    {"name": "Region", "dataType": "string", "sourceColumn": "Region"},
                    {"name": "State", "dataType": "string", "sourceColumn": "State"}
                ],
                "measures": [
                    {
                        "name": "Total Capacities",
                        "expression": "COUNTROWS(Capacities)",
                        "formatString": "#,0"
                    }
                ],
                "partitions": [
                    {
                        "name": "Capacities",
                        "mode": "directLake",
                        "source": {
                            "type": "entity",
                            "entityName": "Capacities",
                            "expressionSource": "DatabaseQuery",
                            "schemaName": "dbo"
                        }
                    }
                ]
            },
            {
                "name": "Workspaces",
                "columns": [
                    {"name": "Id", "dataType": "string", "sourceColumn": "Id"},
                    {"name": "Name", "dataType": "string", "sourceColumn": "Name"},
                    {"name": "Capacity Id", "dataType": "string", "sourceColumn": "Capacity Id"},
                    {"name": "Type", "dataType": "string", "sourceColumn": "Type"},
                    {"name": "State", "dataType": "string", "sourceColumn": "State"}
                ],
                "measures": [
                    {
                        "name": "Total Workspaces",
                        "expression": "COUNTROWS(Workspaces)",
                        "formatString": "#,0"
                    }
                ],
                "partitions": [
                    {
                        "name": "Workspaces",
                        "mode": "directLake",
                        "source": {
                            "type": "entity",
                            "entityName": "Workspaces",
                            "expressionSource": "DatabaseQuery",
                            "schemaName": "dbo"
                        }
                    }
                ]
            },
            {
                "name": "WorkspaceItems",
                "columns": [
                    {"name": "Item Id", "dataType": "string", "sourceColumn": "Item Id"},
                    {"name": "Item Name", "dataType": "string", "sourceColumn": "Item Name"},
                    {"name": "Type", "dataType": "string", "sourceColumn": "Type"},
                    {"name": "Workspace Id", "dataType": "string", "sourceColumn": "Workspace Id"}
                ],
                "measures": [
                    {
                        "name": "Total Items",
                        "expression": "COUNTROWS(WorkspaceItems)",
                        "formatString": "#,0"
                    }
                ],
                "partitions": [
                    {
                        "name": "WorkspaceItems",
                        "mode": "directLake",
                        "source": {
                            "type": "entity",
                            "entityName": "WorkspaceItems",
                            "expressionSource": "DatabaseQuery",
                            "schemaName": "dbo"
                        }
                    }
                ]
            },
            {
                "name": "SemanticModels",
                "columns": [
                    {"name": "Dataset Id", "dataType": "string", "sourceColumn": "Dataset Id"},
                    {"name": "Dataset Name", "dataType": "string", "sourceColumn": "Dataset Name"},
                    {"name": "Workspace Id", "dataType": "string", "sourceColumn": "Workspace Id"},
                    {"name": "Target Storage Mode", "dataType": "string", "sourceColumn": "Target Storage Mode"}
                ],
                "measures": [
                    {
                        "name": "Total Models",
                        "expression": "COUNTROWS(SemanticModels)",
                        "formatString": "#,0"
                    },
                    {
                        "name": "Large Models",
                        "expression": "CALCULATE(COUNTROWS(SemanticModels), SemanticModels[Target Storage Mode] = \"PremiumFiles\")",
                        "formatString": "#,0"
                    }
                ],
                "partitions": [
                    {
                        "name": "SemanticModels",
                        "mode": "directLake",
                        "source": {
                            "type": "entity",
                            "entityName": "SemanticModels",
                            "expressionSource": "DatabaseQuery",
                            "schemaName": "dbo"
                        }
                    }
                ]
            }
        ],
        "relationships": [
            {
                "name": "Workspace-Capacity",
                "fromTable": "Workspaces",
                "fromColumn": "Capacity Id",
                "toTable": "Capacities",
                "toColumn": "Capacity Id"
            },
            {
                "name": "Items-Workspace",
                "fromTable": "WorkspaceItems",
                "fromColumn": "Workspace Id",
                "toTable": "Workspaces",
                "toColumn": "Id"
            },
            {
                "name": "Models-Workspace",
                "fromTable": "SemanticModels",
                "fromColumn": "Workspace Id",
                "toTable": "Workspaces",
                "toColumn": "Id"
            }
        ]
    }
}

try:
    # Create semantic model from BIM definition
    labs.create_semantic_model_from_bim(
        dataset=semantic_model_name,
        bim_file=bim_model,
        lakehouse=lakehouse
    )
    print(f"✓ Semantic model '{semantic_model_name}' created successfully")
except Exception as e:
    print(f"❌ Error creating semantic model: {str(e)}")
    raise

## Step 11: Update Model Connection and Create Report

Connecting the semantic model to the lakehouse and creating the Power BI report.

In [ ]:
import time

print("Waiting for semantic model to be ready...")

check_interval = 5  # Interval in seconds between checks
max_retries = 60  # Maximum number of retries (5 minutes)

retries = 0
model_ready = False

while retries < max_retries:
    try:
        df = fabric.list_datasets()
        df_filt = df[df["Dataset Name"] == semantic_model_name]
        
        if not df_filt.empty:
            print(f"✓ Semantic model found")
            
            # Update DirectLake connection
            print("Updating DirectLake lakehouse connection...")
            labs.directlake.update_direct_lake_model_lakehouse_connection(
                dataset=semantic_model_name,
                lakehouse=lakehouse
            )
            print("✓ Lakehouse connection updated")
            
            # Create basic report
            print(f"Creating Power BI report '{report_name}'...")
            
            # Simple report JSON structure
            report_definition = {
                "config": json.dumps({
                    "version": "5.57",
                    "themeCollection": {
                        "baseTheme": {"name": "CY24SU08", "version": "5.58", "type": 2}
                    },
                    "activeSectionIndex": 0
                }),
                "layoutOptimization": 0,
                "sections": [
                    {
                        "displayName": "Migration Overview",
                        "width": 1280,
                        "height": 720,
                        "displayOption": 1,
                        "visualContainers": []
                    }
                ]
            }
            
            labs_report.create_report_from_reportjson(
                report=report_name,
                dataset=semantic_model_name,
                report_json=report_definition
            )
            
            print(f"✓ Report '{report_name}' created successfully")
            print("\n" + "=" * 70)
            print("✅ SETUP COMPLETE")
            print("=" * 70)
            print(f"Lakehouse: {lakehouse}")
            print(f"Semantic Model: {semantic_model_name}")
            print(f"Report: {report_name}")
            print("\nYou can now open the report and build custom visualizations.")
            print("=" * 70)
            
            model_ready = True
            break
            
    except Exception as e:
        print(f"Attempt {retries + 1}/{max_retries}: {str(e)}")
    
    if not model_ready:
        print(f"Retrying in {check_interval} seconds... ({retries + 1}/{max_retries})")
        time.sleep(check_interval)
        retries += 1

if not model_ready:
    print("❌ Max retries reached. Semantic model may not be ready yet.")
    print("Please check the workspace and try updating the connection manually.")

## Summary

This notebook has:
1. ✅ Validated prerequisites and permissions
2. ✅ Collected comprehensive data from your tenant
3. ✅ Analyzed migration blockers and compatibility issues
4. ✅ Stored all data in a Fabric Lakehouse
5. ✅ Created a semantic model and Power BI report

### Next Steps
1. **Review the Migration Analysis** output above for any blockers or warnings
2. **Open the Power BI Report** to explore your capacity landscape visually
3. **Address any blockers** identified in the analysis
4. **Plan your migration** strategy based on the findings

### Support
For questions or issues, please refer to the deployment guide or contact your Fabric administrator.